In [24]:
!pip install hyperopt

ERROR! Session/line number was not unique in database. History logging moved to new session 60


In [40]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [6]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [7]:
ls

data/       day3.ipynb  day5.ipynb  logs/
day2.ipynb  day4.ipynb  LICENSE     README.md


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1:y_train = to_categorical(y_train)
if y_test.ndim == 1:y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks= [tensorboard_callback]
      )
 
  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),
    
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),
    
    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')     
  ])

In [33]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)


Epoch 1/5
272/272 [==============================] - 7s 26ms/step - loss: 2.5149 - accuracy: 0.3098 - val_loss: 0.8619 - val_accuracy: 0.7331
Epoch 2/5
272/272 [==============================] - 7s 25ms/step - loss: 0.7109 - accuracy: 0.7781 - val_loss: 0.1910 - val_accuracy: 0.9419
Epoch 3/5
272/272 [==============================] - 7s 25ms/step - loss: 0.3626 - accuracy: 0.8885 - val_loss: 0.1203 - val_accuracy: 0.9627
Epoch 4/5
272/272 [==============================] - 7s 25ms/step - loss: 0.2336 - accuracy: 0.9295 - val_loss: 0.0524 - val_accuracy: 0.9857
Epoch 5/5
272/272 [==============================] - 7s 25ms/step - loss: 0.1697 - accuracy: 0.9496 - val_loss: 0.0551 - val_accuracy: 0.9856


0.9564625850340136

In [34]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 0s 4ms/step - loss: 0.1755 - accuracy: 0.9565


0.956462562084198

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),
    
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),
    
    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax')     
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
  )
 
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}


In [54]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 100.0, 'dropout_cnn_block_one': 0.39398411720445914, 'dropout_cnn_block_three': 0.39166278630616136, 'dropout_cnn_block_two': 0.4880504625437576, 'dropout_dense_block_one': 0.3481481253629903, 'dropout_dense_block_two': 0.3631024411541909}
accuracy=0.9437641501426697
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.39451934885728546, 'dropout_cnn_block_three': 0.4187813329818705, 'dropout_cnn_block_two': 0.44807448012443385, 'dropout_dense_block_one': 0.4129640676943784, 'dropout_dense_block_two': 0.5165241754189929}
accuracy=0.9145124554634094
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.4782781652837882, 'dropout_cnn_block_three': 0.31767318955800766, 'dropout_cnn_block_two': 0.41394017038949726, 'dropout_dense_block_one': 0.5583517103947692, 'dropout_dense_block_two': 0.43899931588012864}
accuracy=0.9424036145210266
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.35609036301469643, 'dropout_cnn_block_three': 0.3717603673107025, 'dropout_cnn_block_two': 0.40014743

In [8]:
!echo -e "*\n!.gitignore" > data/.gitignore
!git add data/.gitignore
!git commit -m "add data directory"

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
	modified:   day2.ipynb
	modified:   day3.ipynb
	modified:   day4.ipynb

Untracked files:
	day5.ipynb
	logs/

no changes added to commit


In [0]:
!git config --global user.email "t.laciak78@gmail.com"
!git config --global user.name "Tomek"

In [5]:
ls

drive/  sample_data/


In [1]:
!git add day5.ipynb
!git commit -m "tunning of hyperparams"

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [0]:
!git push -u origin master